# Get Financial Data
Author: Nian

In [1]:
import pandas as pd
import tmdbsimple as tmdb
import os, time, json
from tqdm.notebook import tqdm_notebook

In [2]:
# Key Variables
tmdb_api_secret = 'C:/Users/nianv/.secret/tmdb_api.json'
data_folder = 'Data/'
years_to_get = [year for year in range(2010, 2020)]
api_call_delay = 0.02

In [3]:
with open(tmdb_api_secret, 'r') as f:
    login = json.load(f)
tmdb.API_KEY = login['api-key']

Below functions get_movie (originally get_movie_with_rating) and write_json came from Coding Dojo, adapted as neccessary

In [4]:
def get_movie(movie_id):
    """Adapted from sources https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()

    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## save a "certification" key in info with the certification
            info['certification'] = c['certification']
            break

    return info

In [5]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [6]:
%%time
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    try:
        movie_info = get_movie(movie_id)
        results.append(movie_info)
        
    except Exception as e:
        errors.append((movie_id, e))

pd.DataFrame(results)
pd.DataFrame(errors)

print(f'Results Successfully Retrieved: {len(results)}')
print(f'Results Failed to Retrieve: {len(errors)}')

Results Successfully Retrieved: 3
Results Failed to Retrieve: 1
CPU times: total: 141 ms
Wall time: 2.68 s


In [7]:
# Setup Data Folder
os.makedirs(data_folder, exist_ok=True)
os.makedirs(f'{data_folder}Errors/', exist_ok=True)
os.listdir(data_folder)

['Errors',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [8]:
basics = pd.read_csv(f'{data_folder}title_basics.csv.gz')

In [9]:
for year in tqdm_notebook(years_to_get, desc='YEARS', position=0):
    json_file = f'{data_folder}tmdb_api_results_{year}.json'
    file_exists = os.path.isfile(json_file)
    if not file_exists:
        with open(json_file, 'w') as f:
            json.dump([{'imdb_id':0}], f)
            
    df = basics.loc[basics['startYear']==year].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(json_file)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {year}',
                                  position=1,
                                  leave=True):
        try:
            write_json(get_movie(movie_id), json_file)
            time.sleep(api_call_delay)

        except Exception as e:
            errors.append((movie_id, e))

    results_df = pd.read_json(json_file)
    results_df.to_csv(f'{data_folder}final_tmdb_data_{year}.csv.gz', compression='gzip', index=False)
    errors_df = pd.DataFrame(errors)
    errors_df.to_csv(f'{data_folder}Errors/final_tmdb_data_errors_{year}.csv.gz', compression='gzip', index=False)

    print(f'Results Successfully Retrieved: {results_df.shape[0]}')
    print(f'Results Failed to Retrieve: {errors_df.shape[0]}')
    

YEARS:   0%|          | 0/10 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3873 [00:00<?, ?it/s]

Results Successfully Retrieved: 2747
Results Failed to Retrieve: 1128


Movies from 2011:   0%|          | 0/4235 [00:00<?, ?it/s]

Results Successfully Retrieved: 3042
Results Failed to Retrieve: 2322


...

Results Successfully Retrieved: 4481
Results Failed to Retrieve: 9310


Movies from 2018:   0%|          | 0/5803 [00:00<?, ?it/s]

Results Successfully Retrieved: 4717
Results Failed to Retrieve: 10397


Movies from 2019:   0%|          | 0/5890 [00:00<?, ?it/s]

Results Successfully Retrieved: 4934
Results Failed to Retrieve: 11354
